# 拍點音符紀錄器 for 和弦判斷
- 附點< dots >, 多連音< Tuplet >, 各種拍子、變拍< TimeSig >已處理


In [161]:
import numpy as np
from decimal import *

def convert_chord(tu):
    origin = [0,0,0,0,0,0,0,0,0,0,0,0]
    for i in tu:
        origin[i%12]=1
    return tuple(origin)

# chord = {
#     convert((0,4,7)):'Major',convert((0,3,7)):'minor',
#     convert((0,4,8)):'aug',convert((0,3,6)):'dim',
#     convert((0,4,7,11)):'Major7',convert((0,4,7,10)):'dom7',
#     convert((0,3,7,10)):'minor7',convert((0,3,6,10)):'m7flat5',
#     convert((0,2,7)):'2',convert((0,5,7)):'4',
#     convert((0,2,4,7,10)):'9'
# }
chord = {
    convert((0,4,7)):'C',convert((2,5,9)):'Dm',convert((4,7,11)):'Em',
    convert((0,5,9)):'F',convert((2,7,11)):'G',convert((0,4,9)):'Am',
    convert((2,5,10)):'Bb'
}

chord_np = np.array(chord.keys())

key = {
    0:'C',1:'C-sharp',2:'D',3:'D-sharp',4:'E',5:'F',6:'F-sharp',7:'G',8:'G-sharp',9:'A',10:'A-sharp',11:'B'
} # pitch%12

time = {
    'measure':None,
    'whole':Decimal(1),
    'half':Decimal(1)/2,
    'quarter':Decimal(1)/4,
    'eighth':Decimal(1)/8,
    '16th':Decimal(1)/16,
    '32nd':Decimal(1)/32,
    '64th':Decimal(1)/64,
    '128th':Decimal(1)/128
}

def ifDots(tag):
    if not tag.find('dots'):
        multi = 1
    else:
        dots = tag.find('dots').text
        if dots == '1':
            multi = 1.5
        elif dots == '2':
            multi = 1.75
        elif dots == '3':
            multi = 1.875
    return Decimal(multi)

In [162]:
from bs4 import BeautifulSoup as bs
import sys

#Basket Case是C小調(聽起來像Eb大調)
f = open('C:/Users/BigData/Desktop/You Belong With Me.mscx','r')
music = bs(f.read(),'xml')
f.close()
staff1TimeSig = music.select_one('Score > Staff:nth-of-type(1) TimeSig')
sigN = Decimal(staff1TimeSig.find('sigN').text)
sigD = Decimal(staff1TimeSig.find('sigD').text)
time['measure'] = sigN/sigD
l = []
dic = {}
for staff in music.select('Score > Staff'):
    time_acc = 0
    tupletID = ''
    tupletRatio = 1
    for tag in staff.find_all(['Rest','Chord','Tuplet','TimeSig']):

        #Additional <track> in every measure is ignored
        if (tag.name == 'Chord' or tag.name == 'Rest') and not tag.find('track'):
            multi = ifDots(tag)
            if tag.find('Tuplet') and tag.find('Tuplet').text == tupletID:
                multi *= tupletRatio
            duTime = Decimal(time[tag.select('durationType')[0].text])
            time_acc += duTime * multi
            if tag.name == 'Rest':
                l.append((time_acc, 0))
            else:
                pitches = tag.select('pitch')
                
                #目前音高為手動修改
                addlist = [time_acc] + [int(a.text)-6 for a in pitches]
                l.append(tuple(addlist))
        
        #Only take <Tuplet> with id to avoid children
        #<Tuplet> inside <Chord> or <Rest>
        elif tag.name == 'Tuplet' and tag.get('id'):
            tupletID = tag['id']
            normalNotes = Decimal(tag.find('normalNotes').text)
            actualNotes = Decimal(tag.find('actualNotes').text)
            tupletRatio = normalNotes / actualNotes
        
        elif tag.name == 'TimeSig':
            sigN = Decimal(tag.find('sigN').text)
            sigD = Decimal(tag.find('sigD').text)
            if sigN/sigD != time['measure']:
                time['measure'] = sigN/sigD
    dic.update({'staff_'+staff.get('id') : l})
    l = []

In [87]:
for k in dic:
    print k,dic[k]

staff_4 [(Decimal('1'), 0), (Decimal('2'), 0), (Decimal('3'), 0), (Decimal('4'), 0), (Decimal('5'), 0), (Decimal('6'), 0), (Decimal('7'), 0), (Decimal('8'), 0), (Decimal('9'), 0), (Decimal('10'), 0), (Decimal('11'), 0), (Decimal('12'), 0), (Decimal('13'), 0), (Decimal('14'), 0), (Decimal('15'), 0), (Decimal('16'), 0), (Decimal('17'), 0), (Decimal('18'), 0), (Decimal('19'), 0), (Decimal('20'), 0), (Decimal('21'), 0), (Decimal('22'), 0), (Decimal('23'), 0), (Decimal('24'), 0), (Decimal('25'), 0), (Decimal('26'), 0), (Decimal('27'), 0), (Decimal('28'), 0), (Decimal('29'), 0), (Decimal('30'), 0), (Decimal('31'), 67, 72, 76, 79), (Decimal('32'), 67, 72, 76, 79), (Decimal('33'), 67, 71, 74, 79), (Decimal('34'), 67, 71, 74, 79), (Decimal('35'), 69, 74, 77, 81), (Decimal('36'), 69, 74, 77, 81), (Decimal('37'), 65, 69, 72, 77), (Decimal('38'), 65, 69, 72, 77), (Decimal('39'), 0), (Decimal('40'), 0), (Decimal('41'), 0), (Decimal('42'), 0), (Decimal('43'), 0), (Decimal('44'), 0), (Decimal('45'), 

In [95]:
for i, staff in enumerate(dic, 1):
    print '[Staff #' + str(i) + ']'
    print '-------------------------'
    for e in dic[staff]:
        print e
    print '-------------------------'

[Staff #1]
-------------------------
(Decimal('1'), 0)
(Decimal('2'), 0)
(Decimal('3'), 0)
(Decimal('4'), 0)
(Decimal('5'), 0)
(Decimal('6'), 0)
(Decimal('7'), 0)
(Decimal('8'), 0)
(Decimal('9'), 0)
(Decimal('10'), 0)
(Decimal('11'), 0)
(Decimal('12'), 0)
(Decimal('13'), 0)
(Decimal('14'), 0)
(Decimal('15'), 0)
(Decimal('16'), 0)
(Decimal('17'), 0)
(Decimal('18'), 0)
(Decimal('19'), 0)
(Decimal('20'), 0)
(Decimal('21'), 0)
(Decimal('22'), 0)
(Decimal('23'), 0)
(Decimal('24'), 0)
(Decimal('25'), 0)
(Decimal('26'), 0)
(Decimal('27'), 0)
(Decimal('28'), 0)
(Decimal('29'), 0)
(Decimal('30'), 0)
(Decimal('31'), 67, 72, 76, 79)
(Decimal('32'), 67, 72, 76, 79)
(Decimal('33'), 67, 71, 74, 79)
(Decimal('34'), 67, 71, 74, 79)
(Decimal('35'), 69, 74, 77, 81)
(Decimal('36'), 69, 74, 77, 81)
(Decimal('37'), 65, 69, 72, 77)
(Decimal('38'), 65, 69, 72, 77)
(Decimal('39'), 0)
(Decimal('40'), 0)
(Decimal('41'), 0)
(Decimal('42'), 0)
(Decimal('43'), 0)
(Decimal('44'), 0)
(Decimal('45'), 0)
(Decimal('46'

# 以下未修改

In [172]:
# 第一步，判斷每個staff，有幾成的比例，使用和弦   by Henry
# 1-1. 目前預設，只要大於50%，就紀錄該staff
# 1-2. pitch只要同時有2音以上，就當作和弦
# 1-3. 扣除休止符

pitch_count = 0
percent = 0.5

def have_chord(li):
    return float(len([a for a in li if len(a)>2]))
def have_note(li):
    return float(len([a for a in li if a[1] != 0]))

dominant_percent_staff_list = filter(lambda x: have_chord(dic[x])/have_note(dic[x]) >= percent,dic)

print dominant_percent_staff_list

#測試區

# for staff in dominant_percent_staff_list:
#     print staff
# print dic['staff_4']
# print type(dic)
# for staff in dic:
#     print staff
# for tempo_pitch in dic[staff]:
#     print tempo_pitch
#     print len(tempo_pitch)
# print pitch_count
# print type(pitch_count)
# print len(dic['staff_2'])
# print type(len(dic['staff_2']))
# if float(pitch_count) / float(len(dic['staff_2'])) >= 0.3:
#     dominant_percent_staff_list.append('staff_2')
#     print dominant_percent_staff_list

[u'staff_4', u'staff_5', u'staff_7', u'staff_9', u'staff_11']


In [164]:
import numpy as np
import math
def nearest(nparray, pt):
    n = [math.sqrt(sum((i-pt)**2)) for i in nparray]
    mindis = min(n)
    idx = n.index(mindis)
    return nparray[idx]


In [165]:
chord_dic = {}
for staff in dominant_percent_staff_list:
    chord_gogo = []
    count = 0
    begin = 0
        
    while(count < dic[staff][-1][0]):
        while(count < dic[staff][begin][0]):
            count += 1.0/2
            if dic[staff][begin][1] == 0:
                chord_gogo.append(set())
            else:
                now = [pit%12 for pit in dic[staff][begin][1:]]
#                 closest = nearest(chord_np,convert_chord(tuple(now)))  
#                 chord_gogo.append(chord[tuple(closest)])
                chord_gogo.append(set(now))

        begin += 1
    chord_dic.update({staff : chord_gogo})

In [169]:
for staff in chord_dic:
    print staff, chord_dic[staff]
    print '-----------------------------'
all_chord =  tuple(chord_dic.values())
combined = map(lambda *args: reduce(lambda x,y: x|y,args),*all_chord)
print combined

staff_4 [set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([0, 4, 7]), set([0, 4, 7]), set([0, 4, 7]), set([0, 4, 7]), set([2, 11, 7]), set([2, 11, 7]), set([2, 11, 7]), set([2, 11, 7]), set([9, 2, 5]), set([9, 2, 5]), set([9, 2, 5]), set([9, 2, 5]), set([0, 9, 5]), set([0, 9, 5]), set([0, 9, 5]), set([0, 9, 5]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), set([]), se

In [170]:
final_chord = []
for a in combined:
    print a
    if list(a) == []:
        final_chord.append(0)
    else:
        closest = nearest(chord_np,convert_chord(tuple(a)))
        final_chord.append(chord[tuple(closest)])
print final_chord

set([])
set([])
set([])
set([])
set([0, 7])
set([0, 7])
set([0, 5, 7])
set([0, 7])
set([0, 7])
set([0, 7])
set([0, 7])
set([0, 7])
set([0, 7])
set([0, 7])
set([0, 7])
set([0, 7])
set([2, 7])
set([2, 7])
set([2, 7])
set([2, 7])
set([9, 2])
set([9, 2])
set([9, 2])
set([9, 2])
set([0, 9, 5])
set([0, 9, 5])
set([0, 9, 5])
set([0, 5])
set([0, 7])
set([0, 7])
set([0, 7])
set([0, 7])
set([2, 7])
set([2, 7])
set([2, 7])
set([2, 7])
set([9, 2])
set([9, 2])
set([9, 2])
set([9, 2])
set([0, 9, 5])
set([0, 9, 5])
set([0, 9, 5])
set([0, 9, 5])
set([9, 2])
set([9, 2, 5])
set([0, 9, 5])
set([0, 9, 5])
set([0, 7])
set([0, 4, 7])
set([2, 11, 7])
set([2, 11, 7])
set([9, 2, 5])
set([9, 2, 5])
set([0, 9, 5])
set([0, 9, 5])
set([2, 7])
set([2, 11, 7])
set([2, 11, 7])
set([2, 11, 7])
set([0, 4, 7])
set([0, 4, 7])
set([0, 4, 7])
set([0, 4, 7])
set([0, 2, 4, 7, 11])
set([2, 11, 7])
set([2, 11, 7])
set([2, 11, 7])
set([2, 5, 7, 9, 11])
set([9, 2, 5])
set([9, 2, 5])
set([9, 2, 5])
set([0, 9, 5])
set([0, 9, 5])
s

In [171]:
c = []
for i in range(1,len(final_chord)+1):
    if i%8 == 4:
        c.append(final_chord[i-1])
        print c
        c = []
    else:
        c.append(final_chord[i-1])


[0, 0, 0, 0]
['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']
['C', 'C', 'C', 'C', 'G', 'G', 'G', 'G']
['Dm', 'Dm', 'Dm', 'Dm', 'F', 'F', 'F', 'F']
['C', 'C', 'C', 'C', 'G', 'G', 'G', 'G']
['Dm', 'Dm', 'Dm', 'Dm', 'F', 'F', 'F', 'F']
['Dm', 'Dm', 'F', 'F', 'C', 'C', 'G', 'G']
['Dm', 'Dm', 'F', 'F', 'G', 'G', 'G', 'G']
['C', 'C', 'C', 'C', 'G', 'G', 'G', 'G']
['G', 'Dm', 'Dm', 'Dm', 'F', 'F', 'F', 'F']
['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']
['G', 'G', 'G', 'G', 'Dm', 'Dm', 'Dm', 'Dm']
['F', 'F', 'F', 'F', 'C', 'C', 'C', 'C']
['G', 'G', 'G', 'G', 'Dm', 'Dm', 'Dm', 'Dm']
['F', 'F', 'F', 'F', 'Dm', 'Dm', 'F', 'F']
['C', 'C', 'G', 'G', 'Dm', 'Dm', 'F', 'F']
['G', 'G', 'G', 'G', 'C', 'C', 'C', 'C']
['G', 'G', 'G', 'G', 'G', 'Dm', 'Dm', 'Dm']
['F', 'F', 'F', 'F', 'C', 'C', 'C', 'C']
['G', 'G', 'G', 'G', 'G', 'Dm', 'Dm', 'Dm']
['F', 'F', 'F', 'F', 'C', 'C', 'C', 'C']
['G', 'G', 'G', 'G', 'G', 'Dm', 'Dm', 'Dm']
['F', 'F', 'F', 'F', 'Dm', 'Dm', 'F', 'F']
['C', 'C', 'G', 'G', 'Dm', 'Dm', 'F', 'F']
['C'

In [48]:
l1 = [2,4,5]
l2 = [1,2,3]
l3 = [3,3,4]
a = (l1,l2,l3)
print map(lambda *args: sum(args),*a)

[6, 9, 12]


In [57]:
s1 = set([2,4,5])
s2 = set([1,2,3])
s3 = set([3,3,4])
a = (s1,s2,s3)
print reduce(lambda x,y: x|y,a)
# print map(lambda *args: sum(args),*a)

set([1, 2, 3, 4, 5])


In [70]:
chord_gogo = []
count = 0
begin = 0
while(count<l2[-1][0]):
    while(count < l2[begin][0]):
        count += 1.0/2
        if l2[begin][1] == 0:
            chord_gogo.append(0)
        else:
            chord_gogo.append(key[l2[begin][1]%12])
    begin += 1

[5, 1, 3]


In [ ]:
now = [pit%12 for pit in dic[staff][begin][1:]]
                for pitch in now:
                    poss_key = key[pitch]
                    det = [(n-pitch)%12 for n in now]
                    vec = convert_chord(tuple(det))
                    poss_chord = nearest(chord_np,vec)[0]

In [158]:
s = set()
print s

set([])
